In [1]:
import datetime as dt
import numpy as np
import pandas as pd
from flask import Flask, jsonify
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


In [2]:
app = Flask(__name__)

In [3]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [4]:
Measurement = Base.classes.measurement
Station = Base.classes.station

In [5]:
dict = {"Date": "Prcp"}

In [6]:
@app.route("/")
def welcome():
    return (
        f"Welcome to the Surf API<br/>"
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation"
        f"/api/v1.0/stations"
        f"/api/v1.0/tobs"
        f"/api/v1.0/<start>"
        f"/api/v1.0/<start>/<end>"
    )



In [7]:
@app.route("/api/v1.0/precipitation")
def Precipitation():
    return jsonify(dict)


In [8]:
station_meta = engine.execute('SELECT station FROM Measurement').fetchall()


In [9]:
@app.route("/api/v1.0/stations")
def Stations():
    return jasonify(station_meta)
    

In [10]:
@app.route("/api/v1.0/tobs")
def Tobs():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

    tobs_meta = session.query(Measurement.tobs, Measurement.date).\
        filter(Measurement.date >= prev_year).all()

    observations = []

    for date, tobs in tobs_meta:
        tobs_dict = {}
        tobs_dict['date'] = date
        tobs_dict['tobs'] = tobs
        observations.append(tobs_dict)
    
    return jasonify(observations)


In [11]:
# I have tried another way to solve the problem not sure if this works

@app.route("/api/v1.0/<start>")
def Start_date(start):
    
    modified_start_date = start.replace(date.strftime("%Y%m%d"))
    
    start_date_input = session.query(Measurement.tobs, Measurement.date).\
        filter(Measurement.date >= modified_start_date).all()
    
    start_stats = []
    
    for date, tobs, in start_date_input:
        start_dict = {}
        start_dict['date'] = date
        start_dict['min_tobs'] = min(tobs)
        start_dict['max_tobs'] = max(tobs)
        start_dict['avg_tobs'] = np.mean(tobs)
        start_stats.append(start_dict)
    
    return jasonify(start_stats)

In [12]:
@app.route("/api/v1.0/<start>/<end>")
def calc_temps(start, end):

    range_stats = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start).filter(Measurement.date <= end).all()
    
    range_stats_show = []
    
    for max_tobs, min_tobs, avg_tobs in range_stats:
        range_stats_dict = {'date':'range_stats'}
        range_stats_show.append(range_stats_dict)
        
    return jasonify(range_stats_show)


In [ ]:
if __name__ == "__main__":
    app.run(debug=True)
